In [4]:
import pandas as pd
import numpy as np
import os

# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# acquire
from env import host, user, password
from pydataset import data

<hr style="border-top: 10px groove limegreen; margin-top: 1px; margin-bottom: 1px"></hr>

## Acquire

**<font color=green>A Few Example Methods for Reading Data into Pandas DataFrames</font>**

<hr style="border-top: 10px groove limegreen; margin-top: 1px; margin-bottom: 1px"></hr>

### From a Database

Create your DataFrame using a SQL query to access a database.

**<font color=purple>Use your env file and a handy function to get your connection_url argument.</font>**

```python
from env import host, password, user
```

```python
def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
```
    
```python
sql_query = 'write your sql query here; test it in Sequel Pro first!'
```

```python
pd.read_sql(sql_query, connection_url)
```

**<font color=purple>Put it all together in a single function and throw it into your `acquire.py` file.</font>**

```python
def get_titanic_data():
    return pd.read_sql(query, get_connection('titanic_db'))
```

In [ ]:
# Here is our helper function to get the connection url for our database.

def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [ ]:
# This is a simple query, but they can get much more complicated depending on what we need.

sql_query = 'SELECT * FROM passengers'

In [ ]:
# Here is our acquire function that uses our helper function.

def new_titanic_data():
    return pd.read_sql(sql_query, get_connection('titanic_db'))

In [ ]:
# Use our function to read titanic data into a DataFrame.

titanic_df = get_titanic_data()
titanic_df.head(3)

___

### From Files

- Create your DataFrame from a csv file.

```python
df = pd.read_csv('file_path/file_name.csv')
```
- Create your DataFrame from an AWS S3 file.

```python
df = pd.read_csv('https://s3.amazonaws.com/bucket_and_or_file_name.csv')
```

- Create your DataFrame from a Google sheet using its Share url.

```python
sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'
```  

```python
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
```

```python
df = pd.read_csv(csv_export_url)
```

In [ ]:
# Assign our Google Sheet share url to a variable.

sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'

In [ ]:
# Use the replace method to modify our url, so we can export it.

csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [ ]:
# Use read_csv() method to create our DataFrame.

df_googlesheet = pd.read_csv(csv_export_url)
df_googlesheet.head(2)

___

### From Your Clipboard

Read copy-pasted tabular data and parse it into a DataFrame.

```python
pd.read_clipboard(header=None, names=colums)
```

[Here's](https://towardsdatascience.com/pandas-hacks-read-clipboard-94a05c031382) a short and sweet article that explains it all nicely.

In [ ]:
# Try out the read_clipboard() method here using the article.



___

### From an Excel Sheet

```python
pd.read_excel('your_excel_file_name.xlsx', sheet_name='your_table_name', usecols=['this_one', 'this_one'])
```

In [ ]:
# Read in one sheet from my_telco_churn excel workbook.

customers_df = pd.read_excel('my_telco_churn.xlsx', sheet_name='Table2_CustDetails')
customers_df.head(3)

___

### From Pydataset

Create your DataFrame using Pydataset and Read the Doc.

```python
from pydataset import data

data('iris', show_doc=True)

df_iris = data('iris')
```

In [ ]:
# Create DataFrame using pydataset 'iris'

df_iris = data('iris')
df_iris.head(3)

In [ ]:
# Using Seaborn Datasets. This one has nice column names! :)

iris = sns.load_dataset('iris')
iris.head(3)

<hr style="border-top: 10px groove limegreen; margin-top: 1px; margin-bottom: 1px"></hr>

## Automating Data Acquisition

-  The process of acquiring, preparing, exploring, modeling, and evaluating data is called the Data Science Pipeline.


- As we go through the pipeline, our goal is to end each stage with functions that automate the process and can feed into the next stage, making our work faster and more importantly, repeatable.


- We store our functions from each stage in modules, `acquire.py`, `prepare.py`, etc., and import them for use in our notebooks. All of the helper and main functions are stored in the `.py` file or module.


- Ideally, upon completing the entire process, we should be able to use all of our functions, from each stage, to create one pipeline function that can reproduce our entire process from aquisition to evaluation.


- If our goal is to acquire the titanic data from the Codeup database, both of the funtions below would be stored in an `acquire.py` file and imported into our notebook for use.

<hr style="border-top: 10px groove limegreen; margin-top: 1px; margin-bottom: 1px"></hr>

In [ ]:
# Let's steal our code from above and throw it into a function.

def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [ ]:
def get_titanic_data():
    '''
    This function reads in the titanic data from the Codeup db
    and returns a pandas DataFrame with all columns.
    '''
    sql_query = 'SELECT * FROM passengers'
    return pd.read_sql(sql_query, get_connection('titanic_db'))

<hr style="border-top: 10px groove limegreen; margin-top: 1px; margin-bottom: 1px"></hr>

## Caching Data

**<font color=green>Save time by saving your data to a csv file for future use.</font>**

- Caching or storing data you've retrieved from a database or website makes accessing it later much faster. Basically, cached data reduces load times.

- We can design our acquire functions to get our data for us faster by reading in a csv file, if one exists, and if not, acquiring our data and creating a csv file for later use.

- The `os.path.isfile()` method in Python is used to check whether a specified path is an existing file or not. It returns a boolean value.

<hr style="border-top: 10px groove limegreen; margin-top: 1px; margin-bottom: 1px"></hr>

In [ ]:
# Let's check to see if a file names 'titanic_df.csv' exists in this directory.

os.path.isfile('titanic_df.csv')

In [ ]:
# Let's write our 'titanic_df' DataFrame to a csv file.

titanic_df.to_csv('titanic_df.csv')

In [ ]:
# Let's check again...

os.path.isfile('titanic_df.csv')

- Let's use this concept to write a new function that allows us to hit the Codeup database, write the data to a csv file for later use, and read the data into a pandas DataFrame the next time we call the function and the csv file exists.

In [5]:
# Here is our first helper function that's used below.

def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [6]:
# Let's modify our function from above to cache our data.

def new_titanic_data():
    # Create SQL query.
    sql_query = 'SELECT * FROM passengers'
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('titanic_db'))
    
    # Write DataFrame to a csv file.
    df.to_csv('titanic_df.csv')
    
    return df

In [7]:
def get_titanic_data(cached=False):
    '''
    This function reads in titanic data from Codeup database if cached == False
    or if cached == True reads in titanic df from a csv file, returns df
    '''
    if cached or os.path.isfile('titanic_df.csv') == False:
        df = new_titanic_data()
    else:
        df = pd.read_csv('titanic_df.csv', index_col=0)
    return df

In [8]:
df = get_titanic_data()
df.head(2)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0


In [10]:
df = get_titanic_data(cached=True)
df.head(2)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
